In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 10

In [2]:
G_rwm2_10 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_10.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_10)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_10))

MultiDiGraph with 50000 nodes and 135255 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_10 = list(G_rwm2_10.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_10_df = nx.to_pandas_edgelist(G_rwm2_10) 
rwm2_10_df.head()

,source,target,weight
0,1MsyXEHLuP5FiA99Kn3qKpwrdCmpNt1QDy,1AhzpH2UhXytim46ELJY6sgncpVTVkqAvJ,1.0
1,1MsyXEHLuP5FiA99Kn3qKpwrdCmpNt1QDy,1KKdvgyTXn36pQTsq9UQwcfVcCTL2JYBbH,1.0
2,1GnAQK1cRjTUo8qzHs4hV6L6taDv7gsqaZ,17DymPqjNEv8Ba6KKmsBQ5ZjNEXakd3krz,1.0
3,3Cxge71ZUmfWNA4hDVdZGCnAxaT5DbbbnT,1GD2hUXGgXtmYPFanAkjktAvaSXNdhAJtZ,1.0
4,3Cxge71ZUmfWNA4hDVdZGCnAxaT5DbbbnT,357qDbNztdqY5WF38y56S6uogVmh5bFYN4,1.0


In [8]:
# Number of unique input addresses
rwm2_10_df['source'].nunique()

32043

In [9]:
# Number of unique output addresses
rwm2_10_df['target'].nunique()

31157

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_10_df.describe()

,weight
count,135255.000000
mean,0.781731
std,0.467197
min,-69.648093
25%,0.545759
50%,1.000000
75%,1.000000
max,52.821905


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [11]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_10).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  280


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.56%

## Node Properties

In [12]:
# Get Properties

address = [node for (node, val) in G_rwm2_10.degree()]
degree = [val for (node, val) in G_rwm2_10.degree()]
in_degree = [val for (node, val) in G_rwm2_10.in_degree()]
out_degree = [val for (node, val) in G_rwm2_10.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_10) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [13]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [14]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,14hjFgRXw6TtLsQCNSu5Y9cmbfrSJ1wi2a,1,1,0,0
1,1MsyXEHLuP5FiA99Kn3qKpwrdCmpNt1QDy,6,4,2,0
2,1GnAQK1cRjTUo8qzHs4hV6L6taDv7gsqaZ,2,1,1,0
3,3Cxge71ZUmfWNA4hDVdZGCnAxaT5DbbbnT,3,0,3,0
4,3MRq4uGsQxtYbw65Y5264bXqXCyqNGkcYh,1,1,0,0


In [15]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,5.410200,2.705100,2.705100
std,38.661711,16.932799,25.292302
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,4127.000000,1500.000000,2661.000000


In [16]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,280.000000,280.000000,280.00000
mean,130.203571,51.928571,78.27500
std,462.872949,181.708090,308.03976
min,1.000000,0.000000,0.00000
25%,5.000000,2.000000,1.00000
50%,24.000000,9.500000,7.00000
75%,77.250000,37.000000,30.25000
max,4127.000000,1500.000000,2661.00000


In [17]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49720.000000,49720.000000,49720.000000
mean,4.707422,2.427896,2.279525
std,14.583157,9.451537,8.859482
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,911.000000,905.000000,617.000000


For all 3 cases the degree distribution properties are similar to the overall network. One concern is the presence of isolated nodes.

## Graph Properties

In [18]:
# Density
nx.density(G_rwm2_10) # Connectivity remains very low

5.4103082061641235e-05

In [19]:
# Communities - k-Cliques

com_generator_rwm2_10 = k_clique_communities(G_rwm2_10.to_undirected(), k=10)
com_rwm2_10 = next(com_generator_rwm2_10)
print('The number of communities in the sample : ' + str(len(com_rwm2_10)))

The number of communities in the sample : 10


The sample has a variety of community structures. (Note: Giant Component = 107)